In [2]:
!pip install -U scikit-learn

* Desenvolva o Pipeline completo de um projeto de Aprendizado de Máquina, desde a Análise e preparação dos dados adquiridos até a disponibilização do algoritmo no GitHub.

* é disponibilizado seis (6) arquivos com extensão .npy. Sabe-se que cada arquivo foi adquirido por um sensor, os dados são oriundos de uma máquina com motor elétrico, os quais foram aquisitados a uma taxa de 10 KHz. Os dados de todos os sensores, foram adquiridos de forma simultânea, ou seja, no mesmo instante de tempo. O arquivo Classes.npy possui diferentes estados de funcionamento dessa máquina, podendo ou não possuir diferentes falhas e/ou funcionamento normal.

* Link para o drive com os dados: https://drive.google.com/drive/folders/1ztWi2pHqbbIU-wMMTydq9vssuJmWFPjS

OBS: sensores, são dispositivos capazes de medir grandezas físicas, tem seu funcionamento com base em sinais elétricos, em ambiente industrial podem captar sinais de ruídos e até mesmo entrarem em falha.

In [1]:
import glob
from scipy.io import loadmat
from numpy import asarray
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import scipy
import re
import os
import pandas as pd
import pywt
from scipy.fftpack import fft
from warnings import warn
from sklearn import metrics

import warnings 
warnings.filterwarnings('ignore')

* São 50000 samples
    * São 5 classes e o dataset está balanceado
* cada dado em sensors tem 50000 samples
    * os primeiros 3 possuem 201 valores
    * os últimos 3 possuem 200 valores 

In [3]:
n_sensors = 5
n_samples = 50000
classes = np.load('./dataset/Classes.npy', allow_pickle=True)
classes_k = ['Classe A','Classe B', 'Classe C','Classe D','Classe E']

In [4]:
print(classes.shape)
df = pd.DataFrame(data=classes, columns=["classe"])
print(df['classe'].value_counts())

(50000, 1)
Classe D    10000
Classe A    10000
Classe B    10000
Classe E    10000
Classe C    10000
Name: classe, dtype: int64


# Remoção de NaN:
* Para toda a posição onde há NaN, realiza uma interpolação do sinal e gera os valores
    * Nos sensores 1, 2, 3 e 5, parece ser um valor de vibração
        * Gerei um valor interpolado
    * no Sensor 4, o valor é sempre 50.0 em todo o dataset
        * Se foi falha de captura, esse sensor não tem qualquer informação relevante
        * Se não foi falha de captura, é melhor manter o valor
        * Na dúvida, NaN será colocado como 0 no sinal.


In [5]:
def fill_the_gaps(data):
    ok = np.invert(np.isnan(data))
    xp = ok.ravel().nonzero()[0]
    fp = data[np.invert(np.isnan(data))]
    x  = np.isnan(data).ravel().nonzero()[0]
    data[np.isnan(data)] = np.interp(x, xp, fp)
    return data

In [8]:
sensor_data = []
for i in range(n_sensors):
    sensor_data.append(np.load(f'./dataset/Dados_{i+1}.npy', allow_pickle=True))
for i in [0,1,2,4]:
    for j in range(n_samples):
        sensor_data[i][j] = fill_the_gaps(sensor_data[i][j])

for j in range(n_samples):
    sensor_data[3][j] = np.nan_to_num(sensor_data[3][j], nan=0)

In [9]:
d_map ={}
for ck in classes_k:
    d_map[ck] = []
    for s in range(n_sensors):
        d_map[ck].append(sensor_data[s][(classes==ck).flatten()])

In [13]:
print("Por Classe:")
for ck in classes_k:
    for s in range(n_sensors):
        print(f'{ck}_s{s}: {np.min(np.min(d_map[ck][s]))}, {np.max(np.max(d_map[ck][s]))}')
    print('\n')
print("Por Sensor:")
for i, s in enumerate(sensor_data):
    print(f"s{i}", np.min(np.min(s)), np.max(np.max(s)))


Por Classe:
Classe A_s0: -0.500583548307494, 3.0
Classe A_s1: -1.239735330040627, 5.0
Classe A_s2: -1.4939143838578246, 5.0
Classe A_s3: 0.0, 50.0
Classe A_s4: -19.858602870009083, 139.85894522385962


Classe B_s0: -0.44908397404783623, 5.0
Classe B_s1: -1.188724617411255, 4.0
Classe B_s2: -1.557491418788932, 5.0
Classe B_s3: 0.0, 50.0
Classe B_s4: -19.959756580177167, 139.85206665754947


Classe C_s0: -0.38018984209420753, 5.0
Classe C_s1: -0.8765107971159641, 5.0
Classe C_s2: -1.065159994223182, 5.0
Classe C_s3: 0.0, 50.0
Classe C_s4: -19.9475593405161, 139.99831052645226


Classe D_s0: -0.41449470775403624, 5.0
Classe D_s1: -0.9802582694655667, 5.0
Classe D_s2: -1.4092966639934001, 5.0
Classe D_s3: 0.0, 50.0
Classe D_s4: -19.947602225929288, 139.91842472367367


Classe E_s0: -0.4410823237785372, 5.0
Classe E_s1: -0.8610807389362894, 5.0
Classe E_s2: -1.459181372335915, 5.0
Classe E_s3: 0.0, 50.0
Classe E_s4: -19.964571047700915, 139.8746937044742


Por Sensor:
s0 -0.500583548307494 

* Os ranges por classe não apresentam muita informação útil
* Por sensor, parece ser possível considerar alguns limites:
    * s0, s1 e s2: [-2.0, 5.0]
    * s3: nan ou 50.0
    * s4 -20., 140.
    
    

In [1]:
fig = plt.subplots(figsize=(16, 5))
plt.plot(sensor_data[0][0])
plt.plot(sensor_data[1][0])
plt.plot(sensor_data[2][0])

NameError: name 'plt' is not defined

10 KHz
200 Samples...


In [ ]:
# Function for FFT

def apply_fft(x, fs, num_samples):
    f = np.linspace(0.0, (fs/2.0), num_samples//2)
    freq_values = fft(x)
    freq_values = 2.0/num_samples * np.abs(freq_values[0:num_samples//2])
    return f, freq_values

In [ ]:
from scipy import fftpack
from scipy.signal import butter,filtfilt
sig_noise_fft = scipy.fftpack.fft(sensor_data[0][1])
sig_noise_amp = 2 / time.size * np.abs(sig_noise_fft)
sig_noise_freq = np.abs(scipy.fftpack.fftfreq(time.size, 3/1000))